In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import numpy as np

In [2]:
# Path to the training and validation folders containing image files
training_folder = r"KAPOYA\training"
validation_folder = r"KAPOYA\validation"

# Load labels from csv file
training_label_file = pd.read_csv(os.path.join(training_folder, "output.csv"))
validation_label_file = pd.read_csv(os.path.join(validation_folder, "output.csv"))

# Initialize empty lists to store training and validation images and labels
X_train = []
y_train = []

X_val = []
y_val = []

# Function for loading and preprocessing images with basic progress bar
def load_and_preprocess_images(folder, label_file, X, y):
    total_images = len(label_file)
    for i, row in enumerate(label_file.itertuples(), 1):
        image_path = os.path.join(folder, "images", row.Filename)
        image = cv2.imread(image_path)

        if image is None:
            print(image_path)
            continue

        image = cv2.resize(image, (224, 224))  # Resize image to VGG16 input size
        image = image / 255.0  # Normalize pixel values
        X.append(image)
        y.append(row.Label)

        # Print basic progress bar
        progress = i / total_images
        progress_bar = "=" * int(progress * 20) + "-" * (20 - int(progress * 20))
        print(f"[{progress_bar}] {i}/{total_images}", end="\r")


In [3]:

# Load and preprocess images with basic progress bar
load_and_preprocess_images(training_folder, training_label_file, X_train, y_train)
load_and_preprocess_images(validation_folder, validation_label_file, X_val, y_val)

# Convert lists to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)

print("\nShape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of the base model
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1, activation='linear')

# Create a Sequential model with VGG16 base and custom classification layers
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with progress bar and plot the training and validation loss
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=1)

# Plot training loss and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model on test data
loss, mae = model.evaluate(X_val, y_val)

print("Mean Squared Error:", loss)
print("Mean Absolute Error:", mae)

In [ ]:
# Train the model on the training data
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model on test data
loss, mae = model.evaluate(X_val, y_val)

print("Mean Squared Error:", loss)
print("Mean Absolute Error:", mae)

In [ ]:
# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False)

# Load and preprocess an example image
img_path = 'example_image.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)


In [ ]:

# Get the activations of intermediate layers (feature maps)
layer_outputs = [layer.output for layer in model.layers[1:]]
activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(img_array)


In [ ]:
# Perform Global Average Pooling to convert feature maps to a fixed-length vector
pooled_features = tf.keras.layers.GlobalAveragePooling2D()(activations[-1])

# Define a simple classifier model
classifier = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='linear')
])


In [ ]:

# Make predictions using the classifier
predicted_label = classifier(pooled_features)

# Print the predicted label
print("Predicted Label:", predicted_label)


In [ ]:

# Get the predicted class label
predicted_label = np.argmax(predictions)

# Print the predicted label
print("Predicted Label:", predicted_label)

In [ ]:

# Visualize the feature maps
for i, activation in enumerate(activations):
    plt.figure()
    plt.matshow(activation[0, :, :, 0], cmap='viridis')  # Visualize the feature map of the first channel
    plt.title('Feature Map {}'.format(i))
    plt.show()